# Michelin Rated Restaurants in France - Analysis

In [17]:
# Module import (comprehensive for entire notebook)
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import geopandas as gpd
import mapclassify
import folium
import branca

Importing the processed data

In [18]:
# All restaurants
all_france = pd.read_csv("../../data/France/all_restaurants.csv")

In [19]:
# We define Paris, 75
paris = all_france[all_france['city'] == 'Paris']

# Everything but Paris
rest_of_france = all_france[all_france['city'] != 'Paris']

----
&nbsp;
#### Michelin Rated Restaurants Information

In [20]:
def dataframe_info(data):
    print(f"Unique Departments: {data['department'].nunique()}")
    print(f"Unique Cities: {data['city'].nunique()}")
    print(f"Shape: {data.shape}")
    display(data.head(3))

In [21]:
dataframe_info(all_france)

Unique Departments: 92
Unique Cities: 617
Shape: (1033, 15)


,name,address,city,department_num,department,region,country,price,price_num,cuisine,url,award,stars,longitude,latitude
0,La Vague d'Or - Cheval Blanc St-Tropez,"Plage de la Bouillabaisse, Saint-Tropez, 83990...",Saint-Tropez,83,Var,Provence-Alpes-Côte d'Azur,France,€€€€,4,"Creative, Modern Cuisine",https://www.chevalblanc.com/fr/maison/st-tropez/,3 MICHELIN Stars,3.0,6.626154,43.266585
1,René et Maxime Meilleur,"Hameau de Saint-Marcel, Saint-Martin-de-Belle...",Saint-Martin-de-Belleville,73,Savoie,Auvergne-Rhône-Alpes,France,€€€€,4,"Creative, Regional Cuisine",https://www.la-bouitte.com/fr/,3 MICHELIN Stars,3.0,6.513306,45.369046
2,Kei,"5 rue du Coq-Héron, Paris, 75001, France",Paris,75,Paris,Île-de-France,France,€€€€,4,"Modern Cuisine, Creative",https://www.restaurant-kei.fr/,3 MICHELIN Stars,3.0,2.342285,48.864395


In [22]:
dataframe_info(paris)

Unique Departments: 1
Unique Cities: 1
Shape: (159, 15)


,name,address,city,department_num,department,region,country,price,price_num,cuisine,url,award,stars,longitude,latitude
2,Kei,"5 rue du Coq-Héron, Paris, 75001, France",Paris,75,Paris,Île-de-France,France,€€€€,4,"Modern Cuisine, Creative",https://www.restaurant-kei.fr/,3 MICHELIN Stars,3.0,2.342285,48.864395
11,Le Pré Catelan,"Route de Suresnes - bois de Boulogne, Paris, 7...",Paris,75,Paris,Île-de-France,France,€€€€,4,"Modern Cuisine, Classic Cuisine",https://www.leprecatelan.com/,3 MICHELIN Stars,3.0,2.250718,48.863937
13,Épicure,"Le Bristol, 112 rue du Faubourg-Saint-Honoré, ...",Paris,75,Paris,Île-de-France,France,€€€€,4,Modern Cuisine,https://www.oetkercollection.com/fr/hotels/le-...,3 MICHELIN Stars,3.0,2.314625,48.871762


----
&nbsp;
## Analysis of Michelin Stars by department and region

### `top_restaurants_by_department` function

Imported below and defined in `functions.py`, this function analyses a dataset of restaurants and outputs the top $n$ departments with the highest count of restaurants with a specified Michelin star rating.

Usage:
```python
top_restaurants_by_department(data, star_rating, top_n, display_restaurants=True)
```
----
&nbsp;
`data` is a pandas DataFrame that contains restaurant information. It expects the following columns: 'department', 'stars', 'name', 'city', and 'department_num'.

`star_rating` is an integer (1, 2, or 3) specifying the Michelin star rating of interest.

`top_n` is an integer specifying the number of departments to consider.

`display_restaurants` is an optional boolean flag (default: True), Prints the name and city.

`display_info` is an optional boolean flag (default: False). Prints style of cuisine and url

In [31]:
from functions import top_restaurants_by_department

Top 3 departments by three star restaurants

In [42]:
# excludes Paris
top_restaurants_by_department(rest_of_france, 3, 3, display_info=True)

Top 3 departments with most ⭐⭐⭐ restaurants:


Department: Bouches-du-Rhône (13)
4 ⭐⭐⭐ Restaurants

Restaurant: AM par Alexandre Mazzia
Location: Marseille
Style of Cuisine: Creative
URL: https://www.alexandre-mazzia.com/

Restaurant: Le Petit Nice
Location: Marseille
Style of Cuisine: Seafood
URL: https://www.passedat.fr

Restaurant: La Villa Madie
Location: Cassis
Style of Cuisine: Creative, Mediterranean Cuisine
URL: https://lavillamadie.com/

Restaurant: L'Oustau de Baumanière
Location: Les Baux-de-Provence
Style of Cuisine: Creative
URL: https://www.baumaniere.com/



Department: Haute-Savoie (74)
2 ⭐⭐⭐ Restaurants

Restaurant: Flocons de Sel
Location: Megève
Style of Cuisine: Modern Cuisine
URL: https://www.floconsdesel.com/fr/

Restaurant: Le Clos des Sens
Location: Annecy
Style of Cuisine: Creative
URL: https://www.closdessens.com/



Department: Savoie (73)
2 ⭐⭐⭐ Restaurants

Restaurant: Le 1947 à Cheval Blanc
Location: Courchevel
Style of Cuisine: Creative
URL: https://www.ch

In [33]:
# We specify 'top5' however, the paris dataset has 1 department; 75
top_restaurants_by_department(paris, 3, 5, display_info=True)

⭐⭐⭐ Restaurants in Paris

Department: Paris (75)
9 ⭐⭐⭐ Restaurants

Restaurant: Kei
Location: Paris
Style of Cuisine: Modern Cuisine, Creative
URL: https://www.restaurant-kei.fr/

Restaurant: Le Pré Catelan
Location: Paris
Style of Cuisine: Modern Cuisine, Classic Cuisine
URL: https://www.leprecatelan.com/

Restaurant: Épicure
Location: Paris
Style of Cuisine: Modern Cuisine
URL: https://www.oetkercollection.com/fr/hotels/le-bristol-paris/restaurants-et-bar/epicure/

Restaurant: Arpège
Location: Paris
Style of Cuisine: Creative
URL: https://www.alain-passard.com/

Restaurant: L'Ambroisie
Location: Paris
Style of Cuisine: Classic Cuisine
URL: https://www.ambroisie-paris.com/

Restaurant: Le Cinq
Location: Paris
Style of Cuisine: Modern Cuisine
URL: https://www.fourseasons.com

Restaurant: Plénitude - Cheval Blanc Paris
Location: Paris
Style of Cuisine: Creative
URL: https://www.chevalblanc.com/fr/maison/paris/

Restaurant: Pierre Gagnaire
Location: Paris
Style of Cuisine: Creative
URL: 

### `top_restaurants_by_region` function

Again, defined in `functions.py`, it analyses a dataset of restaurants and outputs the top $n$ departments with the highest count of restaurants with a specified Michelin star rating. This iteration allows the argument 'all' for `star_rating` and `top_n`

In [34]:
from functions import top_restaurants_by_region

In [35]:
# For brevity, the display_restaurants flag is set False
# Using 'all' parameter for star_rating
top_restaurants_by_region(rest_of_france, 'all', 5, display_restaurants=False)

Top 5 regions with most starred restaurants:


Region: Auvergne-Rhône-Alpes
95 Starred Restaurants

8 ⭐⭐⭐ Restaurants

15 ⭐⭐ Restaurants

72 ⭐ Restaurants



Region: Provence-Alpes-Côte d'Azur
77 Starred Restaurants

6 ⭐⭐⭐ Restaurants

5 ⭐⭐ Restaurants

66 ⭐ Restaurants



Region: Nouvelle-Aquitaine
57 Starred Restaurants

1 ⭐⭐⭐ Restaurant

7 ⭐⭐ Restaurants

49 ⭐ Restaurants



Region: Grand Est
51 Starred Restaurants

1 ⭐⭐⭐ Restaurant

9 ⭐⭐ Restaurants

41 ⭐ Restaurants



Region: Occitanie
49 Starred Restaurants

1 ⭐⭐⭐ Restaurant

8 ⭐⭐ Restaurants

40 ⭐ Restaurants





----
&nbsp;
## Geographical Data Analysis

Importing data

In [38]:
departments = pd.read_csv("../../data/France/department_restaurants.csv")
print(f"Departments: {departments.shape[0]}")
departments.head()

Departments: 96


,code,geometry,department,region,bib_gourmand,1_star,2_star,3_star,total_stars,total_restaurants,population,population_density(pop/area),area(sq_km),locations
0,01,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",Ain,Auvergne-Rhône-Alpes,6,4,0,1,7,11,652432,114.2,5713.06,"{1: [(46.3626315, 6.0258533), (46.0070045, 5.3..."
1,02,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",Aisne,Hauts-de-France,0,0,0,0,0,0,531345,71.9,7390.06,"{1: None, 2: None, 3: None}"
2,03,"POLYGON ((3.03207 46.79491, 3.03424 46.7908, 3...",Allier,Auvergne-Rhône-Alpes,4,2,0,0,2,6,335975,45.7,7351.75,"{1: [(46.1246273, 3.4192286), (46.3233297, 2.6..."
3,04,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,1,3,0,0,3,4,164308,23.9,6874.81,"{1: [(43.8303211, 5.7973728), (44.0930379, 6.0..."
4,05,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",Hautes-Alpes,Provence-Alpes-Côte d'Azur,2,1,0,0,1,3,141220,25.3,5581.82,"{1: [(44.7067072, 6.6078486)], 2: None, 3: None}"


In [39]:
regions = pd.read_csv("../../data/France/region_restaurants.csv")
print(f"Regions: {regions.shape[0]}")
regions.head()

Regions: 13


,geometry,region,bib_gourmand,1_star,2_star,3_star,total_stars,total_restaurants,population,area(sq_km),population_density(pop/area),locations
0,POLYGON ((2.5905242793946224 49.07965484673242...,Île-de-France,46,110,15,9,167,180,12262544,11969.29,1024.50,"{1: [(48.8652933, 2.3421907), (48.8583566, 2.2..."
1,POLYGON ((2.8746252064056805 47.52042311253978...,Centre-Val de Loire,23,20,1,0,22,44,2573180,39169.91,65.69,"{1: [(47.3602993, 1.7438252), (47.641088, 1.44..."
2,"POLYGON ((3.629423683496518 46.74945902193754,...",Bourgogne-Franche-Comté,20,28,6,1,43,55,2805580,47874.19,58.60,"{1: [(46.7127086, 4.6941996), (46.3034378, 4.8..."
3,POLYGON ((-1.1196225402660513 49.3555679898469...,Normandie,26,20,0,0,20,46,3325032,29892.65,111.23,"{1: [(49.4410412, 1.094415), (48.6468685, -0.5..."
4,"POLYGON ((4.047972977986833 49.40564229475753,...",Hauts-de-France,9,15,2,0,19,26,6004947,31864.47,188.45,"{1: [(50.5221852, 1.5914671), (50.6752627, 3.0..."
